In [ ]:
"""
Script to automatically set up RO & RW guest collections on the AARNet Globus hosts specified in GLOBUS_HOSTS
There is logic to prevent the creation of duplicate guest collections, and the standard test files are copied into the RO guest collection.
The script assumes that each endpoint has exactly one storage gateway and exactly one private mapped collection.
Anyone with write permissions to the private mapped collections can run this script.
"""

import globus_sdk
from globus_sdk import TransferClient, TransferAPIError
from globus_sdk.scopes import TransferScopes
from globus_sdk.globus_app import UserApp
from pprint import pprint

# UserApp Client ID - <replace this with your own UserApp client UUID>
NATIVE_CLIENT_ID = "<replace this with your own UserApp client UUID>"

GLOBUS_HOSTS = [
    "apl-node1.globus.aarnet.net.au",
    "apl-node2.globus.aarnet.net.au",
    "apl-node3.globus.aarnet.net.au",
]

AARNET_DEMONSTRATORS_GROUP_ID = "f2fdf716-592f-11ef-888e-b34a8e234634"

# AARNet Sydney Read-only Test Collection (may go away soon)
# https://app.globus.org/file-manager/collections/12b88024-167c-4869-ab40-69bbf07ce3c5
SRC_COLLECTION = "12b88024-167c-4869-ab40-69bbf07ce3c5"
SRC_PATH = "/standard_test_files/5MB-in-tiny-files"
DST_PATH = "/5MB-in-tiny-files"

# # Could also use original files from CERN (slower)
# # https://app.globus.org/file-manager/collections/722751ce-1264-43b8-9160-a9272f746d78
# SRC_COLLECTION = "722751ce-1264-43b8-9160-a9272f746d78"
# SRC_PATH = "/5MB-in-tiny-files"
# DST_PATH = "/5MB-in-tiny-files"


def main():
    user_app = UserApp("create-guest-collections", client_id=NATIVE_CLIENT_ID)
    # pprint(user_app.__dict__)
    for globus_host in GLOBUS_HOSTS:
        gcs_client = globus_sdk.GCSClient(globus_host, app=user_app)
        # pprint(gcs_client.__dict__)
        endpoint_id = gcs_client._endpoint_client_id

        storage_gateways = list(gcs_client.get_storage_gateway_list())
        # pprint(storage_gateways)

        if len(storage_gateways) != 1:
            print(f"Unable to determine unique storage gateway for {globus_info['hostname']}")
            exit(1)

        storage_gateway_id = storage_gateways[0]['id']

        collections = list(gcs_client.get_collection_list())
        # pprint(collections)

        mapped_collections = [collection for collection in collections if collection['collection_type'] == 'mapped' and not collection['public']]
        if len(storage_gateways) != 1:
            print(f"Unable to determine unique private mapped collection for {globus_info['hostname']}")
            exit(1)

        mapped_collection = mapped_collections[0]

        transfer_client = TransferClient(app=user_app).add_app_data_access_scope(mapped_collection['id'])
        # pprint(transfer_client.__dict__)

        guest_collections = [
            collection for collection in collections
            if collection['collection_type'] == 'guest'
            and collection['public']
            and collection['mapped_collection_id'] == mapped_collection['id']
            ]

        # pprint(guest_collections)

        attach_data_access_scope(gcs_client, mapped_collection['id'])


        # # Comment out this line if the mapped collection is high assurance
        # attach_data_access_scope(gcs_client, globus_info['mapped_collection_id'])

        ensure_user_credential(gcs_client, storage_gateway_id)

        for collection_type in ['Public RO Guest', 'Public RW Guest']:
            display_name=mapped_collection['display_name'].replace('Private Mapped', collection_type)
            collection_base_path = f"/{collection_type.replace(' ', '_')}_Collection"  # RO_TEST or RW_TEST (this is a little ugly)

            try:
                response = transfer_client.operation_mkdir(mapped_collection['id'], collection_base_path)
                # pprint(response)
                print(f'Created directory "{collection_base_path}" in collection "{mapped_collection["display_name"]}"')
            except TransferAPIError as e:
                # pprint(e.__dict__)
                if e.code == 'ExternalError.MkdirFailed.Exists':
                    print(f'Directory "{collection_base_path}" already exists in collection "{mapped_collection["display_name"]}"')
                else:
                    raise e

            collection_request = globus_sdk.GuestCollectionDocument(
                public=True,
                collection_base_path=collection_base_path,
                display_name=display_name,
                mapped_collection_id=mapped_collection['id'],
            )
            # pprint(collection_request)

            found_collections = [
                collection for collection in collections 
                if collection['collection_type'] == 'guest' 
                and collection['public']
                and collection['display_name'] == display_name
                ]

            if found_collections:
                print(f'Public guest collection "{display_name}" already exists.')
                if len(found_collections) == 1:
                    collection = found_collections[0]
                else:
                    print(f'Unable to determine unique match for collection "{display_name}"')
                    exit(1)

                # Unable to update collection_base_path (immutable), so we can't change that
                print(f'Updating existing guest collection "{display_name}" with collection ID {collection["id"]}')
                gcs_client.update_collection(collection['id'], collection_request)

            else:
                collection = gcs_client.create_collection(collection_request)
                print(f'Created new guest collection "{display_name}" with collection ID {collection["id"]}')

            # Add read permissions for all authenticated users on all collections
            rule_data = {
                'DATA_TYPE': 'access',
                'path': '/',
                'permissions': 'r',
                'principal': '',
                'principal_type': 'all_authenticated_users',
            }
            try:
                result = transfer_client.add_endpoint_acl_rule(collection["id"], rule_data)
                # pprint(result)
                print('Added "r" permissions for "AARNet Demonstrators" group')
            except TransferAPIError as e:
                # pprint(e.__dict__)
                if e.code == 'Exists':
                    print('Permissions already exist for "all authenticated users"')
                else:
                    raise e

            # Add read/write permissions for "AARNet Demonstrators" group on RW collection
            if 'RW' in collection_type:
                rule_data = {
                    'DATA_TYPE': 'access',
                    'path': '/',
                    'permissions': 'rw',
                    'principal': AARNET_DEMONSTRATORS_GROUP_ID,
                    'principal_type': 'group',
                }
                try:
                    result = transfer_client.add_endpoint_acl_rule(collection["id"], rule_data)
                    # pprint(result)
                    print('Added "rw" permissions for "AARNet Demonstrators" group')
                except TransferAPIError as e:
                    # pprint(e.__dict__)
                    if e.code == 'Exists':
                        print('Permissions already exist for "AARNet Demonstrators" group')
                    else:
                        raise e

            # Transfer standard test files into RO collection
            if 'RO' in collection_type:
                try:
                    response = transfer_client.operation_mkdir(collection["id"], DST_PATH) # Error if directory already exists
                    # pprint(response)
                    print(f'Created directory "{DST_PATH}" in collection "{collection["display_name"]}"')

                    transfer_request = globus_sdk.TransferData(
                        source_endpoint=SRC_COLLECTION,
                        destination_endpoint=collection["id"],
                    )
                    transfer_request.add_item(SRC_PATH, DST_PATH)

                    task = transfer_client.submit_transfer(transfer_request)
                    print(f"Submitted transfer of standard test files to {DST_PATH}. Task ID: {task['task_id']}.")
                except TransferAPIError as e:
                    # pprint(e.__dict__)
                    if e.code == 'ExternalError.MkdirFailed.Exists':
                        print(f'Directory "{DST_PATH}" already exists in collection "{collection["display_name"]}"')
                    else:
                        raise e


def attach_data_access_scope(gcs_client, collection_id):
    """Compose and attach a ``data_access`` scope for the supplied collection"""
    endpoint_scopes = gcs_client.get_gcs_endpoint_scopes(gcs_client.endpoint_client_id)
    collection_scopes = gcs_client.get_gcs_collection_scopes(collection_id)

    manage_collections = globus_sdk.Scope(endpoint_scopes.manage_collections)
    data_access = globus_sdk.Scope(collection_scopes.data_access, optional=True)

    manage_collections.add_dependency(data_access)

    gcs_client.add_app_scope(manage_collections)


def ensure_user_credential(gcs_client, storage_gateway_id):
    """
    Ensure that the user has a user credential on the client.
    This is the mapping between Globus Auth (OAuth2) and the local system's permissions.
    """
    # Depending on the endpoint & storage gateway, this request document may need to
    # include more complex information such as a local username.
    # Consult with the endpoint owner for more detailed info on user mappings and
    # other particular requirements.
    req = globus_sdk.UserCredentialDocument(storage_gateway_id=storage_gateway_id)
    try:
        gcs_client.create_user_credential(req)
    except globus_sdk.GCSAPIError as err:
        # A user credential already exists, no need to create it.
        if err.http_status != 409:
            raise

if __name__ == '__main__':
    main()

Directory "/Public_RO_Guest_Collection" already exists in collection "AARNet Globus Endpoint NSW (ARTM) POSIX Gateway Private Mapped Collection"
Public guest collection "AARNet Globus Endpoint NSW (ARTM) POSIX Gateway Public RO Guest Collection" already exists.
Updating existing guest collection "AARNet Globus Endpoint NSW (ARTM) POSIX Gateway Public RO Guest Collection" with collection ID 9e472d3a-ac18-42d0-bac8-3c9220801fbe
Permissions already exist for "all authenticated users"
Directory "/standard_test_files" already exists in collection "AARNet Globus Endpoint NSW (ARTM) POSIX Gateway Public RO Guest Collection"
Directory "/Public_RW_Guest_Collection" already exists in collection "AARNet Globus Endpoint NSW (ARTM) POSIX Gateway Private Mapped Collection"
Public guest collection "AARNet Globus Endpoint NSW (ARTM) POSIX Gateway Public RW Guest Collection" already exists.
Updating existing guest collection "AARNet Globus Endpoint NSW (ARTM) POSIX Gateway Public RW Guest Collection" w